In [3]:
import numpy as np 
import glob
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import *
from keras.layers.merge import concatenate
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


# Data processing

In [20]:
# make config
rows = 256
cols = 256
train_path = "train"
label_path = "label"
img_type = "jpg"
test_path = "test"
npy_path = "npydata"

In [19]:
def create_data(data_path, npy_path, rows, cols):   
    imgs = glob.glob("data\\" + data_path + "\\*." + img_type)
    i = 0
    img_count = len(imgs)
    imgs_data = np.ndarray((img_count, rows, cols, 1), dtype=np.uint8)
    for img_full_name in imgs:
        img_name = img_full_name[img_full_name.rindex("\\") + 1:]
        # test
        img = load_img("data\\" + data_path + "\\" + img_name, grayscale = True)
        img = img_to_array(img)
        imgs_data[i] = img
        if i % 10 == 0:
            print('Done: {0}/{1} images'.format(i, img_count))
        i += 1
    print('Created.')
    np.save("data\\" + npy_path + '\\imgs_' + data_path + '.npy', imgs_data)
    print('Saving to .npy files done.')

In [18]:
def load_data(data_path, npy_path, process_colors = False):
    imgs = np.load("data\\" + npy_path + '\\imgs_' + data_path + '.npy')
    imgs = imgs.astype('float32')
    imgs /= 255
    if (process_colors):
        imgs[imgs > 0.5] = 1
        imgs[imgs <= 0.5] = 0
    return imgs

# U-net

In [29]:
def get_model(rows, cols):

    input_shape = (rows, cols, 1)

    model = Sequential()
    model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, 3, padding='same', activation='relu'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))

    model.add(Conv2D(1, 3, padding='same', activation='sigmoid'))

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [16]:
def train(train_path, test_path, label_path, npy_path, rows, cols):
    imgs_train = load_data(train_path, npy_path)
    imgs_test = load_data(test_path, npy_path)
    imgs_mask_train = load_data(label_path, npy_path, True)
    model = get_model(rows, cols)

    model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit(imgs_train, imgs_mask_train, batch_size=4, nb_epoch=5, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

    imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
    save_results(imgs_mask_test)

In [23]:
def predict(test_path, npy_path, rows, cols):
    imgs_test = load_data(test_path, npy_path)
    model = get_model(rows, cols)
    model.load_weights('unet.hdf5')
    imgs_mask_test = model.predict(imgs_test, batch_size = 4, verbose=1)
    save_results(imgs_mask_test)

In [26]:
def save_results(imgs):
    i = 1
    for img in imgs:
            img = array_to_img(img)
            img.save("data\\results\\%d.jpg"%(i))   
            i += 1

# Main

In [ ]:
create_data(train_path, npy_path, rows, cols)
create_data(label_path, npy_path, rows, cols)
create_data(test_path, npy_path, rows, cols)

In [30]:
train(train_path, test_path, label_path, npy_path, rows, cols)

C:\Users\artem.eprev\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 115 samples, validate on 29 samples
Epoch 1/5
115/115 [==============================] - 341s 3s/step - loss: 0.5579 - acc: 0.8483 - val_loss: 0.3135 - val_acc: 0.9548

Epoch 00001: loss improved from inf to 0.55791, saving model to unet.hdf5
Epoch 2/5
115/115 [==============================] - 340s 3s/step - loss: 0.3419 - acc: 0.8494 - val_loss: 0.0835 - val_acc: 0.9548

Epoch 00002: loss improved from 0.55791 to 0.34189, saving model to unet.hdf5
Epoch 3/5
115/115 [==============================] - 366s 3s/step - loss: 0.1909 - acc: 0.8494 - val_loss: 0.0761 - val_acc: 0.9548

Epoch 00003: loss improved from 0.34189 to 0.19093, saving model to unet.hdf5
Epoch 4/5
115/115 [==============================] - 386s 3s/step - loss: 0.1660 - acc: 0.8494 - val_loss: 0.0619 - val_acc: 0.9548

Epoch 00004: loss improved from 0.19093 to 0.16603, saving model to unet.hdf5
Epoch 5/5
115/115 [==============================] - 346s 3s/step - loss: 0.1428 - acc: 0.9346 - val_loss: 0.0577 -

In [31]:
predict(test_path, npy_path, rows, cols)

5/5 [==============================] - 5s 902ms/step
